# Demo notebook for slide 3.2

In [1]:
%matplotlib widget

In [2]:
import bmcs_utils

In [3]:
from bmcs_matmod.slide.slide_explorer import SlideExplorer
import bmcs_utils.api as bu

In [4]:
se = SlideExplorer()

Parameters for elementary convergence studies - no physical relevance

Parameters rendering a stress-strain response comparable with a `MARS` model - example concrete

# Debugging LDM implementation

In [6]:
material_params = dict(
    E_s=1, gamma_s=1, K_s=1, S_s=0.5, c_s=1, bartau=1, 
    E_w=1, S_w=0.5, c_w = 1, m = 0.05, f_t=1, f_c=20, f_c0=10, eta=0.0
)

Critical step size - step smaller than the elastic range in tension.

In [7]:
def get_step_cr(**kw):
    f_t = kw['f_t']
    E_w = kw['E_w']
    return f_t/E_w
d_w = get_step_cr(**material_params)
w_max = 10
n_steps = int(w_max / d_w)
n_steps

10

In [12]:
%%capture
se = SlideExplorer(n_steps=200, k_max=50)
se.slide_model.trait_set(**material_params)
# se.trait_set(s_x_1 = 2, s_y_1 = 0, w_1 = 0);
# se.run()
# se.trait_set(s_x_1 = 2, s_y_1 = 1, w_1 = 0);
# se.run()
se.trait_set(s_x_1 = 2, s_y_1 = 1, w_1 = 10);
se.run()

In [13]:
se.energy_dissipation.show_iter = False
se.inel_state_evolution.t_max=3.0
bu.InteractiveWindow([se.inel_state_evolution, se, se.energy_dissipation, 
                      se.slide_model], figsize=(9,4), tight_layout=False).interact()